In [1]:
import json
import random
import pandas as pd
from datasets import load_dataset
from datetime import datetime

In [ ]:
#this function takes in all our data and returns the correct data of only tweets created within date range,
#printing to daterange.json

#parse to make sure tweets are correct range
def filter_tweets_by_date_range(tweets, start_date, end_date):
    filtered_tweets = []
    for tweet in tweets:
        tweet_created_at = datetime.strptime(tweet['tweet_created_at']['$date'], '%Y-%m-%dT%H:%M:%S.%fZ')
        if start_date <= tweet_created_at <= end_date:
            filtered_tweets.append(tweet)
    return filtered_tweets

def subsample_json(input_file, output_file, start_date, end_date):
    with open(input_file, 'r') as f:
        data = json.load(f)
    # Filter tweets based on date range
    filtered_data = filter_tweets_by_date_range(data, start_date, end_date)
    with open(output_file, 'w') as f:
        json.dump(filtered_data, f, indent=4)
        print('awesome')
        
input_json_file = 'Jew.json'
output_json_file = 'daterange.json'
start_date = datetime(2022, 9, 1)
end_date = datetime(2022, 11, 1)
subsample_json(input_json_file, output_json_file, start_date, end_date)
print('success')

In [2]:
with open('daterange.json', 'r', encoding='utf-8') as f:  # Specify the encoding
        data = json.load(f)
        df = pd.DataFrame(data)

df = df[df["pure_text"].notna()]

# Print the DataFrame
print(df)

# Export the DataFrame to a new CSV file
df.to_csv('pure_text_dr.csv', index=False)

                                         _id  \
0       {'$oid': '64258bae2643a0bff73ba28a'}   
1       {'$oid': '642063ebf6d089e30a4d67a6'}   
2       {'$oid': '642053ea43f20e2970dc6c5c'}   
3       {'$oid': '64259f7d359f5e3f5a481d4c'}   
4       {'$oid': '64258baf2643a0bff73d886a'}   
...                                      ...   
241968  {'$oid': '642630c71b4dd40e9592337d'}   
241969  {'$oid': '642063bb4e764554ef4ec4eb'}   
241970  {'$oid': '64208c374e764554efb251f1'}   
241971  {'$oid': '64205ea6d4f06de39731fa56'}   
241972  {'$oid': '64262f7f71f61b5f0f92009b'}   

                                                pure_text  \
0       RT @Tracking_Power:When we say “Zionists” they...   
1       RT @Tracking_Power:When we say “Zionists” they...   
2       RT @MarsMonkey_:🧢, even Jesus (and apostoles) ...   
3       RT @RightWingCope:Kanye also claimed Disney wa...   
4       RT @Tracking_Power:When we say “Zionists” they...   
...                                                   ...

In [9]:
# #this function samples o.1 percent of daterange.json and returns in randomsample.json


# def subsample(input_file, output_file, subsample_percent):
#     with open(input_file, 'r', encoding='utf-8') as f:  # Specify the encoding
#         data = json.load(f)
    
#     subsample_size = int(len(data) * (subsample_percent / 100))
    
#     subsample = random.sample(data, subsample_size)
    
#     with open(output_file, 'w', encoding='utf-8') as f:  # Specify the encoding
#         json.dump(subsample, f, indent=4, ensure_ascii=False)  # Ensure_ascii=False for non-ASCII characters

# # Usage example
# input_json_file = 'daterange.json' 
# output_json_file = 'randomsample.json'  
# subsample_percent = 0.1  # Replace with the desired subsample percentage

# subsample(input_json_file, output_json_file, subsample_percent)


In [13]:
#this function creates the annotated data set

# Load JSON dataset
file_path = 'daterange.json'
prototype_tweets = load_dataset('json', data_files=file_path)
data = prototype_tweets['train']

# Convert the data to a Pandas DataFrame
df = pd.DataFrame(data)
print(len(df))
df = df[df["pure_text"].notna()]
print(len(df))
df['label'] = None
df = df.sample(n=700, random_state=42)
df = df.reset_index(drop=True)
df['row_number'] = df.index + 1
print(df['tweet_created_at'])
df = df[["row_number","rt_text","pure_text", "label"]]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

241973
110368
0      {'$date': '2022-10-25T01:08:16.000Z'}
1      {'$date': '2022-10-27T00:42:53.000Z'}
2      {'$date': '2022-09-07T19:02:36.000Z'}
3      {'$date': '2022-10-10T17:44:17.000Z'}
4      {'$date': '2022-09-19T03:49:09.000Z'}
                       ...                  
695    {'$date': '2022-09-02T01:44:01.000Z'}
696    {'$date': '2022-10-12T22:04:23.000Z'}
697    {'$date': '2022-09-30T09:41:51.000Z'}
698    {'$date': '2022-09-27T05:54:47.000Z'}
699    {'$date': '2022-09-26T18:30:59.000Z'}
Name: tweet_created_at, Length: 700, dtype: object


In [14]:
#prints annotated data set to cv to be annotated
csv_file_path = 'prototype_tweets.csv'
df.to_csv(csv_file_path, index=False)